<a href="https://colab.research.google.com/github/Jlauf-MBAPMP/NewGitTest/blob/master/Copy_of_NaiveBayes_Lab_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy matplotlib scikit-learn ipywidgets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import CategoricalNB
from google.colab import output
output.enable_custom_widget_manager()

def create_dataset():
    # Create a dataset with two binary features
    X = np.random.randint(2, size=(100, 2))
    y = np.logical_xor(X[:, 0], X[:, 1]).astype(int)
    return X, y

def train_naive_bayes(X, y):
    model = CategoricalNB()
    model.fit(X, y)
    return model

# Create and train the initial model
X, y = create_dataset()
model = train_naive_bayes(X, y)

In [ ]:
def plot_decision_boundary(X, y, model):
    plt.figure(figsize=(10, 8))

    # Get predictions for all points
    predictions = model.predict(X)

    # Plot points
    for class_value in [0, 1]:
        X_class = X[predictions == class_value]
        color = 'red' if class_value == 1 else 'green'
        plt.scatter(X_class[:, 0] + np.random.normal(0, 0.05, X_class.shape[0]),
                    X_class[:, 1] + np.random.normal(0, 0.05, X_class.shape[0]),
                    color=color, alpha=0.5,
                    label=f'Class {"Sick" if class_value == 1 else "Not Sick"}')

    # Plot decision boundary
    for x1 in [0, 1]:
        for x2 in [0, 1]:
            prob = model.predict_proba([[x1, x2]])[0]
            predicted_class = 1 if prob[1] > 0.5 else 0
            color = 'red' if predicted_class == 1 else 'green'
            plt.text(x1, x2, f'P(Sick|X)={prob[1]:.2f}\nP(Not Sick|X)={prob[0]:.2f}',
                     ha='center', va='center',
                     bbox=dict(facecolor='white', alpha=0.5, edgecolor=color))

    plt.xlim(-0.5, 1.5)
    plt.ylim(-0.5, 1.5)
    plt.xticks([0, 1])
    plt.yticks([0, 1])
    plt.xlabel("Smoker")
    plt.ylabel("Exercises")
    plt.title("Naive Bayes Decision Boundary (Categorical Features)")
    plt.legend()
    plt.show()


# Plot the initial decision boundary
plot_decision_boundary(X, y, model)

#Adversarial Examples with Naive Bayes

## Do Adversarial Examples exist with Naive Bayes?

When would Adversial Examples exist with Naive Bayes?

If they do what does this imply about the data in the feature group collection?


#Data Poisoning with Naive Bayes

##You have to change the probabilities to poison Naive Bayes

Which feature combinations would be easier to poison (flip the class prediction in a feature combination: [0, 0], [0, 1], [1, 0], [1, 1])?

In [ ]:
from ipywidgets import interact, interactive, fixed
from ipywidgets import widgets

class InteractivePlot:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.model = train_naive_bayes(self.X, self.y)
        self.fig, self.ax = plt.subplots(figsize=(10, 8))
        self.update_plot()

    def update_plot(self):
        self.ax.clear()

        # Get current predictions for all possible feature combinations
        all_combinations = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
        predictions = self.model.predict(all_combinations)
        probabilities = self.model.predict_proba(all_combinations)

        # Plot points
        for i, (x1, x2) in enumerate(all_combinations):
            mask = (self.X[:, 0] == x1) & (self.X[:, 1] == x2)
            points = self.X[mask]
            if len(points) > 0:
                jittered_points = points + np.random.normal(0, 0.05, points.shape)
                color = 'red' if predictions[i] == 1 else 'green'
                label = f'Class {"Sick" if predictions[i] == 1 else "Not Sick"} at features: [{x1},{x2}]'
                self.ax.scatter(jittered_points[:, 0], jittered_points[:, 1], color=color, alpha=0.5, label=label)

        # Plot decision boundary
        for i, (x1, x2) in enumerate(all_combinations):
            prob = probabilities[i]
            color = 'red' if predictions[i] == 1 else 'blue'
            self.ax.text(x1, x2, f'P(C=1|X)={prob[1]:.2f}\nP(C=0|X)={prob[0]:.2f}',
                         ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5, edgecolor=color))


        self.ax.set_xlim(-0.5, 1.5)
        self.ax.set_ylim(-0.5, 1.5)
        self.ax.set_xticks([0, 1])
        self.ax.set_yticks([0, 1])
        self.ax.set_xlabel("Smoker")
        self.ax.set_ylabel("Exercises")
        self.ax.set_title("Interactive Naive Bayes Decision Boundary")
        self.ax.legend()
        plt.close(self.fig)
        display(self.fig)

    def add_point(self, x, y, label):
        self.X = np.vstack((self.X, [x, y]))
        self.y = np.append(self.y, label)
        self.model = train_naive_bayes(self.X, self.y)
        self.update_plot()

# Create interactive plot
interactive_plot = InteractivePlot(X, y)

# Create interactive widgets
x_widget = widgets.Dropdown(options=[0, 1], description='Is Smoker:')
y_widget = widgets.Dropdown(options=[0, 1], description='Does Exercise:')
label_widget = widgets.Dropdown(options=[('Sick', 1), ('Not Sick', 0)], description='Class:')
add_button = widgets.Button(description='Add Point')

def on_button_click(b):
    interactive_plot.add_point(x_widget.value, y_widget.value, label_widget.value)

add_button.on_click(on_button_click)

# Display widgets
display(widgets.VBox([x_widget, y_widget, label_widget, add_button]))

Use the Add Point button to add poisoned points with a specified label to specific feature combinations

# Model Inversion

Model Inversion takes place when the attacker would like to determine what the input is for a known output label.

## What is a way that an attacker could figure out the valid inputs for a known label in Naive Bayes?

#Model Inference

Is there anything we can learn from the behavior of the model to determine any characteristics of the models such as the hyperparameter values, number of layers, if they are using Dropout, etc.

# Training Dataset Leakage

Are there any signs that the inputs provided to the model were part of the training set. One way to identify the use of trainging data as inputs to the model is when output confidence scores are significantly higher for input values when providing training data as input to the model?

Could you identify training data by iterating through all the possible input feature values that can be passed to the model?

#Model Stealing

Model Stealing involves copying the model to obtaining the model internal configuration. In the case of Naive Bayes how would it be similar to model stealing with kNN.

Based on the answer above how would one steal the model?